# Training using SageMaker Managed Spot Training

The example here is almost the same as previous labs we run for cifar10 dataset, where we trained a Keras Sequential model on SageMaker. The model used for this proplem was a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

This notebook tackles the exact same problem with the same solution, but it has been modified to be able to run using SageMaker Managed Spot infrastructure. SageMaker Managed Spot uses [EC2 Spot Instances](https://aws.amazon.com/ec2/spot/) to run Training at a lower cost.

Please read the original notebook and try it out to gain an understanding of the ML use-case and how it is being solved. We will not delve into that here in this notebook.


## Set up the environment

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [3]:
bucket = '<BUCKET NAME>'
prefix = 'sagemaker/script-mode-spot'

## Download the CIFAR-10 dataset
Downloading the test and training data takes around 5 minutes.

In [4]:
#!pip install wget
# import wget # for TF2

In [5]:
!mkdir data
!python generate_cifar10_tfrecords_v2.py --data-dir data/

mkdir: cannot create directory 'data': File exists
Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
data/
100% [..................................................] 170498071 / 170498071Generating data//train/train.tfrecords
Generating data//validation/validation.tfrecords
Generating data//eval/eval.tfrecords
Done!


### Uploading the data to s3

In [6]:
dataset_location = sagemaker_session.upload_data(bucket=bucket, path='data', key_prefix=prefix)
display(dataset_location)

's3://demo-saeed/sagemaker/script-mode-spot'

### Configuring hyperparameters and metrics
SageMaker can get training metrics directly from the logs and send them to CloudWatch metrics.

In [7]:
keras_metric_definition = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - acc: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - acc: ([0-9\\.]+).*'},
    {'Name': 'validation:accuracy', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: ([0-9\\.]+).*'},
    {'Name': 'validation:loss', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_acc: [0-9\\.]+.*'},
    {'Name': 'sec/steps', 'Regex': '.* - \d+s (\d+)[mu]s/step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: [0-9\\.]+'}
]
hyperparameters = {'epochs': 10, 'batch-size' : 256}

# Managed Spot Training with a TensorFlow Estimator

For Managed Spot Training using a TensorFlow Estimator we need to configure two things:
1. Enable the `train_use_spot_instances` constructor arg - a simple self-explanatory boolean.
2. Set the `train_max_wait` constructor arg - this is an int arg representing the amount of time you are willing to wait for Spot infrastructure to become available. Some instance types are harder to get at Spot prices and you may have to wait longer. You are not charged for time spent waiting for Spot infrastructure to become available, you're only charged for actual compute time spent once Spot instances have been successfully procured.

Normally, a third requirement would also be necessary here - modifying your code to ensure a regular checkpointing cadence - however, TensorFlow Estimators already do this, so no changes are necessary here. Checkpointing is highly recommended for Manage Spot Training jobs due to the fact that Spot instances can be interrupted with short notice and using checkpoints to resume from the last interruption ensures you don't lose any progress made before the interruption.

Feel free to toggle the `train_use_spot_instances` variable to see the effect of running the same job using regular (a.k.a. "On Demand") infrastructure.

Note that `train_max_wait` can be set if and only if `train_use_spot_instances` is enabled and **must** be greater than or equal to `train_max_run`.

In [9]:
train_use_spot_instances = True
train_max_run=3600
train_max_wait = 7200 if train_use_spot_instances else None

In [10]:
from sagemaker.tensorflow import TensorFlow


source_dir = os.path.join(os.getcwd(), 'source_dir')
# Location where results of model training are saved.
model_artifacts_location = 's3://{}/{}/artifacts'.format(bucket,prefix)

estimator = TensorFlow(base_job_name='spot-cifar10-tf',
                       entry_point='cifar10_keras_main.py',
                       source_dir=source_dir,
                       output_path=model_artifacts_location,
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type='ml.p3.2xlarge',
                       metric_definitions=keras_metric_definition,
                       train_use_spot_instances=train_use_spot_instances,
                       train_max_run=train_max_run,
                       train_max_wait=train_max_wait)

In this example, we set ```wait=False``` if you want to see the output logs, change this to ```wait=True```

In [11]:
remote_inputs = {'train' : dataset_location+'/train', 'validation' : dataset_location+'/validation', 'eval' : dataset_location+'/eval'}
estimator.fit(remote_inputs, wait=True)

2020-06-23 20:43:44 Starting - Starting the training job...
2020-06-23 20:43:48 Starting - Launching requested ML instances.........
2020-06-23 20:45:34 Starting - Preparing the instances for training......
2020-06-23 20:46:41 Downloading - Downloading input data...
2020-06-23 20:47:07 Training - Downloading the training image...
2020-06-23 20:47:29 Training - Training image download completed. Training in progress.2020-06-23 20:47:32,101 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-23 20:47:32,522 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperpara

# Savings
Towards the end of the job you should see two lines of output printed:

- `Training seconds: X` : This is the actual compute-time your training job spent
- `Billable seconds: Y` : This is the time you will be billed for after Spot discounting is applied.

If you enabled the `train_use_spot_instances` var then you should see a notable difference between `X` and `Y` signifying the cost savings you will get for having chosen Managed Spot Training. This should be reflected in an additional line:
- `Managed Spot Training savings: (1-Y/X)*100 %`